# ASR
## finetune

* 训练启动类： class U2Trainer(Trainer)： paddlespeech/s2t/training/trainer.py
* 训练基类： paddlespeech/s2t/training/trainer.py:  def before_train(self): from_scratch = self.resume_or_scratch() 
    * before_train 中控制resume 或者从头开始训练
* def resume_or_scratch(self)： If ``args.checkpoint_path`` is not None, load the checkpoint, else resume training.

## API client

* https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/demos/speech_recognition/README.md

In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
import paddle
from paddlespeech.cli.asr import ASRExecutor

asr_executor = ASRExecutor()
text = asr_executor(
    model='conformer_wenetspeech',
    lang='zh',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='./speech.wav',
    force_yes=False,
    device=paddle.get_device())
print('ASR Result: \n{}'.format(text))

ModuleNotFoundError: No module named 'paddlespeech'

# TTS

## all trained model list 
* https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/demos/text_to_speech/README_cn.md

## official tutorial
* https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/tutorial/tts/tts_tutorial.ipynb
  * 语音合成基本知识 图解
  * 新训练模型调用
  * 前端详解样例
* [Models introduction](https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/source/tts/models_introduction.md)
* [Chinese Rule-Based Text Frontend](https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/source/tts/zh_text_frontend.md)

In [1]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-zh-cv7_css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(model, cfg)

text = "您好，这是试运行。"

sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)

c:\Users\73915\.conda\envs\open_editor\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\73915\.conda\envs\open_editor\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\73915\.conda\envs\open_editor\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\73915\.conda\envs\open_editor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 10 files: 100%|██████████| 10/10 [00:00<?, ?it/s]


TypeError: 'TTSTransformerModel' object is not subscriptable

In [ ]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(model, cfg)

text = "Hello, this is a test run."

sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)


## Mixed Chinese and English TTS: PaddleSpeech

* https://github.com/PaddlePaddle/PaddleSpeech/tree/develop/examples/zh_en_tts/tts3


## TTS 模型checkpoints 文件说明

* default.yaml 存储用于训练模型的配置。
* snapshot_iter_*.pdz 是检查点文件，其中*是它经过训练的步骤。
* *_stats.npy 是特征的统计文件，如果它在训练前已被标准化。
* phone_id_map.txt 是音素到音素 ID 的映射关系。
* tone_id_map.txt 是在训练声学模型之前分割音调和拼音时，音调到音调 ID 的映射关系。（例如在 csmsc/speedyspeech 的示例中）
* spk_id_map.txt 是多发音人声学模型中 "发音人" 到 "spk_ids" 的映射关系。

## Chinese Rule-Based Text Frontend作用

* https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/source/tts/zh_text_frontend.md
* 作用预处理
* 其它引用： https://github.com/speechio/chinese_text_normalization

## TTS papers

* https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/source/tts/tts_papers.md

## 速度控制

* https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/demos/style_fs2/style_syn.py
  * durations_scale

In [2]:
# 中文frontend
from paddlespeech.t2s.frontend.zh_normalization.text_normlization import TextNormalizer
from paddlespeech.t2s.utils.error_rate import char_errors
text_normalizer = TextNormalizer()

raw_text = "只要有33/4的人同意，就可以通过决议。"
textnorm_text = text_normalizer.normalize_sentence(raw_text)
print(textnorm_text)

# mix frantend

from paddlespeech.t2s.frontend.mix_frontend import MixFrontend, MixTextProcessor

phone_id_map = "D:/code/audio/PaddleSpeech/examples/zh_en_tts/tts3/dump/phone_id_map.txt"
# tone_vocab_path = 

sentence = "只要有33/4的人同意，就可以通过决议。this is true"
d_inputs = MixTextProcessor.get_dom_split(sentence)
tone_vocab_path=None

frontend = MixFrontend(phone_vocab_path=phone_id_map, tone_vocab_path=tone_vocab_path)
print(d_inputs)

只要有四分之三十三的人同意，就可以通过决议。


[2023-06-20 19:58:45,383] [    INFO] - Already cached C:\Users\73915\.paddlenlp\models\bert-base-chinese\bert-base-chinese-vocab.txt
[2023-06-20 19:58:45,402] [    INFO] - tokenizer config file saved in C:\Users\73915\.paddlenlp\models\bert-base-chinese\tokenizer_config.json
[2023-06-20 19:58:45,404] [    INFO] - Special tokens file saved in C:\Users\73915\.paddlenlp\models\bert-base-chinese\special_tokens_map.json


['只要有33/4的人同意，就可以通过决议。this is true']


In [8]:
# 如何得到phone: frontend.phoneticize
frontend.get_input_ids(sentence)

from paddlespeech.t2s.frontend import English

n_frontend = English(phone_vocab_path=phone_id_map)
n_frontend.phoneticize("CEO hats money")

['<s>',
 'S',
 'IY1',
 'OW0',
 ' ',
 'HH',
 'AE1',
 'T',
 'S',
 ' ',
 'M',
 'AH1',
 'N',
 'IY0',
 '</s>']

## voice cloning

* paddlespeech/t2s/exps/voice_cloning.py 提供VectorExecutor和LSTMSpeakerEncoder两种方法, 模型都需要从其它地方预训练好。
* VectorExecutor 使用的是： speaker verification中模（getting a speaker embedding from an audio）
  * 原理：使用一个声纹识别模型，对每段音频生成speaker embedding（preprocess中生成到dump/embed中，见examples/aishell3/vc2），作为Fastspeech2的输入
  * https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/demos/speaker_verification/README.md
  * 声纹识别模型：https://github.com/PaddlePaddle/PaddleSpeech/tree/develop/examples/voxceleb/sv0
    * https://arxiv.org/abs/2005.07143
  * 步骤
    * 1. preprocess data: pitch, stats, duration, energy, and also speaker embedding (参考examples/aishell3/vc2/local/preprocess.sh中使用paddlespeech/t2s/exps/fastspeech2/vc2_infer.py 对每个wav文件转换)
    * 2. 训练：--voice-cloning 这个参数设置为True （这样就不是使用speaker_id_map.txt跟模型一起训练speaker embedding），使用步骤1中得到的speaker embedding
    * 3. 

In [11]:
# !wget -c https://paddlespeech.bj.bcebos.com/vector/audio/85236145389.wav

# !wget -c https://paddlespeech.bj.bcebos.com/vector/audio/123456789.wav

# 如何生成声纹，speaker embedding

from paddlespeech.cli.vector import VectorExecutor
import paddle

vector_executor = VectorExecutor()
audio_emb = vector_executor(
    model='ecapatdnn_voxceleb12',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='bark_generation_long_cn_speak_0.wav',
    device=paddle.get_device())
print('Audio embedding Result: \n{}'.format(audio_emb))

test_emb = vector_executor(
    model='ecapatdnn_voxceleb12',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='bark_generation_long_cn_speak_1.wav',
    device=paddle.get_device())
print('Test embedding Result: \n{}'.format(test_emb))

# score range [0, 1]
score = vector_executor.get_embeddings_score(audio_emb, test_emb)
print(f"Eembeddings Score: {score}")

[2023-06-21 09:40:47,306] [ WARNING] - Not regular input, please input again
[2023-06-21 09:40:47,999] [ WARNING] - Not regular input, please input again
100%|██████████| 259820/259820 [01:58<00:00, 2200.40it/s]


Audio embedding Result: 
[  3.827975     0.13272521  -4.137721    11.970211    -0.6367527
   3.91727    -12.058795    -5.835077    -1.4045581   -2.7857635
  -8.94035      2.2840116    3.8795233    1.9127965    0.30097526
  11.097014     0.7374594   -4.4722733   -1.7190295   13.2098465
  -6.5891705    2.6714993    7.1824784  -12.26595      1.1015433
  -3.7810457   10.272412     2.9668195  -12.617983     1.6873999
   0.01952853  -2.6424098   -6.725892   -14.130116     4.6985292
  -8.085005     7.8202457    2.697162     3.9581048    6.700255
   1.8994429    0.40445453   0.42131668  14.750108     1.1762869
  -2.396632    -3.5896065   -8.093888     6.7898555   -4.3486342
   5.057675     3.657227    -7.0912824    2.3937678   -3.9378397
   0.8221358   -4.3861146    0.7883899    4.8804474    2.12671
  -2.3923247   -0.80586314   6.40859      3.7341008    1.490904
   0.26913887   4.262577     1.2481384    3.7242131  -13.616583
  -1.0505577   -4.00911      0.5632631   -0.6302681  -11.49861
  -9.8

## example code


In [22]:
from paddlespeech.cli.tts.infer import TTSExecutor
tts_excutor = TTSExecutor()
from IPython.display import Audio

lang='mix'
am='fastspeech2_mix'
voc='pwgan_csmsc'
out_path = f'./{am}_{voc}_{lang}.wav'

text="你好，欢迎使用Paddle Speech中英文混合TTS功能，开始你的合成之旅吧!"

tts_excutor(text=text, lang=lang, am=am, voc=voc, spk_id=175, output=out_path) # type: ignore

[2023-06-21 22:38:06,735] [    INFO] - Already cached C:\Users\73915\.paddlenlp\models\bert-base-chinese\bert-base-chinese-vocab.txt
[2023-06-21 22:38:06,754] [    INFO] - tokenizer config file saved in C:\Users\73915\.paddlenlp\models\bert-base-chinese\tokenizer_config.json
[2023-06-21 22:38:06,756] [    INFO] - Special tokens file saved in C:\Users\73915\.paddlenlp\models\bert-base-chinese\special_tokens_map.json


'd:\\code\\learn\\pylearn\\pybasic\\sound\\fastspeech2_mix_pwgan_csmsc_mix.wav'

In [24]:
from IPython.display import Audio
res = tts_excutor._outputs['wav'].numpy()
Audio(res.flatten(), rate=tts_excutor.am_config.fs)
# tts_excutor.am_config.fs
# res.shape

In [37]:
# TTS 类想要按照字母念出来，必须加入空格。
text="你好，欢迎使用Paddle Speech中英文混合T T S功能，开始你的合成之旅吧!"
tts_excutor(text=text, lang=lang, am=am, voc=voc, spk_id=175, output=out_path) # type: ignore
res = tts_excutor._outputs['wav'].numpy()
Audio(res.flatten(), rate=tts_excutor.am_config.fs)

In [8]:
# 如何使用自定义训练产生的模型

from paddlespeech.cli.tts.infer import TTSExecutor
tts_excutor_custom = TTSExecutor()
from IPython.display import Audio

lang='mix'
am='fastspeech2_mix'
voc=[
    'pwgan_csmsc',
    'pwgan_ljspeech',
    'pwgan_aishell3',
    'pwgan_vctk',
    'mb_melgan_csmsc',
    'style_melgan_csmsc',
    'hifigan_csmsc',
    'hifigan_ljspeech',
    'hifigan_aishell3',
    'hifigan_vctk',
    'wavernn_csmsc',
    'pwgan_male',
    'hifigan_male',
][-3]
    
out_path = f'./{am}_{voc}_{lang}.wav'

text="你好，欢迎使用Paddle Speech中英文混合TTS功能，开始你的合成之旅吧!"

# tts_excutor(text=text, lang=lang, am=am, voc=voc, spk_id=175, output=out_path) # type: ignore
tts_excutor_custom(text=text, 
    am_ckpt="d:/code/audio/PaddleSpeech/examples/zh_en_tts/tts3/exp/default/checkpoints/snapshot_iter_164084.pdz", # type: ignore
    am_config="d:/code/audio/PaddleSpeech/examples/zh_en_tts/tts3/conf/default.yaml", # type: ignore
    # am_stat="", # type: ignore
    phones_dict="d:/code/audio/PaddleSpeech/examples/zh_en_tts/tts3/dump/phone_id_map.txt", # type: ignore
    speaker_dict="d:/code/audio/PaddleSpeech/examples/zh_en_tts/tts3/dump/speaker_id_map.txt", # type: ignore
    spk_id=174, output=out_path, lang=lang, am=am, voc=voc,
)

res = tts_excutor_custom._outputs['wav'].numpy()
Audio(res.flatten(), rate=tts_excutor_custom.am_config.fs)

100%|██████████| 48939/48939 [00:20<00:00, 2343.77it/s]
[2023-06-22 00:25:53,420] [    INFO] - Already cached C:\Users\73915\.paddlenlp\models\bert-base-chinese\bert-base-chinese-vocab.txt
[2023-06-22 00:25:53,438] [    INFO] - tokenizer config file saved in C:\Users\73915\.paddlenlp\models\bert-base-chinese\tokenizer_config.json
[2023-06-22 00:25:53,440] [    INFO] - Special tokens file saved in C:\Users\73915\.paddlenlp\models\bert-base-chinese\special_tokens_map.json


In [44]:
tts_excutor_custom(text=text, 
    spk_id=174, output=out_path, lang=lang, am=am, voc=voc,
)
res = tts_excutor_custom._outputs['wav'].numpy()
Audio(res.flatten(), rate=tts_excutor_custom.am_config.fs)

# easymms

In [ ]:
from easymms.models.tts import TTSModel

tts = TTSModel('eng')
res = tts.synthesize("This is a simple example")
tts.save(res)

In [5]:
from IPython.display import Audio
Audio(res[0], rate=res[1])

In [ ]:
tts_cn = TTSModel("cmn")